## OCI Document Understanding による同期OCRサンプルコード
本サンプルコードは処理の流れをご理解いただくことを目的としているためエラー処理は省かれています。

### 関連ドキュメント
[API のリファレンス](https://docs.oracle.com/en-us/iaas/api/#/en/document-understanding/20221109/)

SDK のドキュメント
- [AIServiceDocumentClient](https://docs.oracle.com/en-us/iaas/tools/python/2.141.0/api/ai_document/client/oci.ai_document.AIServiceDocumentClient.html)
- [AIServiceDocumentClient.analyze_document()](https://docs.oracle.com/en-us/iaas/tools/python/latest/api/ai_document/client/oci.ai_document.AIServiceDocumentClient.html#oci.ai_document.AIServiceDocumentClient.analyze_document)
- [models.AnalyzeDocumentDetails()](https://docs.oracle.com/en-us/iaas/tools/python/latest/api/ai_document/models/oci.ai_document.models.AnalyzeDocumentDetails.html)
    - [models.DocumentFeature()](https://docs.oracle.com/en-us/iaas/tools/python/latest/api/ai_document/models/oci.ai_document.models.DocumentFeature.html#oci.ai_document.models.DocumentFeature)
        - [models.DocumentTextExtractionFeature()](https://docs.oracle.com/en-us/iaas/tools/python/latest/api/ai_document/models/oci.ai_document.models.DocumentTextExtractionFeature.html#oci.ai_document.models.DocumentTextExtractionFeature)
    - [models.DocumentDetails()](https://docs.oracle.com/en-us/iaas/tools/python/latest/api/ai_document/models/oci.ai_document.models.DocumentDetails.html#oci.ai_document.models.DocumentDetails)
        - [models.InlineDocumentDetails()](https://docs.oracle.com/en-us/iaas/tools/python/2.141.0/api/ai_document/models/oci.ai_document.models.InlineDocumentDetails.html#oci.ai_document.models.InlineDocumentDetails)
- [models.AnalyzeDocumentResult()](https://docs.oracle.com/en-us/iaas/tools/python/latest/api/ai_document/models/oci.ai_document.models.AnalyzeDocumentResult.html#oci.ai_document.models.AnalyzeDocumentResult)
    - [models.Page()](https://docs.oracle.com/en-us/iaas/tools/python/latest/api/ai_document/models/oci.ai_document.models.Page.html#oci.ai_document.models.Page)

## パッケージのインストール
OCI の SDK と dotenv パッケージをインストールします。


In [ ]:
%pip install oci
%pip install python-dotenv

### パッケージの import

In [2]:
import os
import base64
import oci
from dotenv import load_dotenv


### 環境変数の設定
.env ファイルに定義された変数を環境変数に設定します。ここでは、OCI のコンパートメントID(OCI_COMPARTMENT_ID) を設定します。

本ノートブックと同じディレクトリに .env_example ファイルの書式で 予め .env ファイルを作成してください。

In [ ]:
load_dotenv()
print(f'OCI_COMPARTMENT_ID = {os.getenv("OCI_COMPARTMENT_ID")}')

### 解析対象画像と解析結果の出力先の定義

In [4]:
# 解析する画像の定義
target_file="images/冷凍食品2.JPG"
# OCRで抽出したデデータの出力先ファイル（json形式）の定義
output_file = "output/" + os.path.splitext(os.path.basename(target_file))[0] + ".json"

### OCI 認証設定
- CONFIG_PROFILE：構成ファイルに定義されたプロファイル名
- config : SDK and Tool Configuration（認証に関する構成情報を定義するディクショナリー）[リファレンス](https://docs.oracle.com/en-us/iaas/tools/python/latest/configuration.html)

参考ドキュメント
- [SDKおよびCLIの構成ファイル](https://docs.oracle.com/ja-jp/iaas/Content/API/Concepts/sdkconfig.htm)
- [Configuration](https://docs.oracle.com/en-us/iaas/tools/python/latest/configuration.html)

In [5]:
CONFIG_PROFILE = "DEFAULT" # 構成ファイルに合わせて変更してください。
config = oci.config.from_file(file_location='~/.oci/config', profile_name=CONFIG_PROFILE)

### OCI Document Understanding サービスクライアントの初期化

参考ドキュメント

- [AIServiceDocumentClient](https://docs.oracle.com/en-us/iaas/tools/python/latest/api/ai_document/client/oci.ai_document.AIServiceDocumentClient.html)

In [6]:
ai_document_client = oci.ai_document.AIServiceDocumentClient(config)

### 画像ファイルの準備

In [7]:
# 画像ファイルを読み込んでbase64エンコード
with open(target_file, 'rb') as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

### 分析の実行
参考ドキュメント
- [AIServiceDocumentClient.analyze_document()](https://docs.oracle.com/en-us/iaas/tools/python/latest/api/ai_document/client/oci.ai_document.AIServiceDocumentClient.html#oci.ai_document.AIServiceDocumentClient.analyze_document)
- [models.AnalyzeDocumentDetails()](https://docs.oracle.com/en-us/iaas/tools/python/latest/api/ai_document/models/oci.ai_document.models.AnalyzeDocumentDetails.html)
    - [models.DocumentFeature()](https://docs.oracle.com/en-us/iaas/tools/python/latest/api/ai_document/models/oci.ai_document.models.DocumentFeature.html#oci.ai_document.models.DocumentFeature)
        - [models.DocumentTextExtractionFeature()](https://docs.oracle.com/en-us/iaas/tools/python/latest/api/ai_document/models/oci.ai_document.models.DocumentTextExtractionFeature.html#oci.ai_document.models.DocumentTextExtractionFeature)
- [models.DocumentDetails()](https://docs.oracle.com/en-us/iaas/tools/python/latest/api/ai_document/models/oci.ai_document.models.DocumentDetails.html#oci.ai_document.models.DocumentDetails)
    - [models.InlineDocumentDetails()](https://docs.oracle.com/en-us/iaas/tools/python/2.141.0/api/ai_document/models/oci.ai_document.models.InlineDocumentDetails.html#oci.ai_document.models.InlineDocumentDetails)

In [8]:
analyze_document_response = ai_document_client.analyze_document(
    analyze_document_details=oci.ai_document.models.AnalyzeDocumentDetails(
        features=[
            #oci.ai_document.models.DocumentKeyValueExtractionFeature(
            oci.ai_document.models.DocumentTextExtractionFeature(
                feature_type="TEXT_EXTRACTION") # テキスト抽出機能を指定する
        ],
        document=oci.ai_document.models.InlineDocumentDetails(
            source="INLINE", # インラインドキュメントを指定する
            data=encoded_string), # 画像ファイルをbase64エンコードした文字列を指定する
        compartment_id=os.getenv("OCI_COMPARTMENT_ID"), # コンパートメントIDを指定する
    )
)


### 分析結果の取得とファイル出力
参考ドキュメント
- [models.AnalyzeDocumentResult()](https://docs.oracle.com/en-us/iaas/tools/python/latest/api/ai_document/models/oci.ai_document.models.AnalyzeDocumentResult.html#oci.ai_document.models.AnalyzeDocumentResult)
    - [models.Page()](https://docs.oracle.com/en-us/iaas/tools/python/latest/api/ai_document/models/oci.ai_document.models.Page.html#oci.ai_document.models.Page)

In [ ]:
# レスポンスからOCRデータを取得
ocr_result = analyze_document_response.data
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(str(ocr_result))

print(f"OCR結果を {output_file} に保存しました。")

### ページ数の表示

In [ ]:
print(len(ocr_result.pages))

### 入力画像のディメンジョンの表示（高さ、幅、単位）
（この例では1ページ目（インデックスが"0"）のみを表示）

In [ ]:
print(ocr_result.pages[0].dimensions.unit)
print(ocr_result.pages[0].dimensions.height)
print(ocr_result.pages[0].dimensions.width)




### 抽出した行数の表示
（この例では1ページ目（インデックスが"0"）に含まれる行数のみを表示）

In [ ]:
print(len(ocr_result.pages[0].lines))

### 抽出した行のテキストの表示
(この例では1ページ目（インデックスが"0"）のすべての行の信頼度スコアとテキストを表示)

In [ ]:
for i, line in enumerate(ocr_result.pages[0].lines):
    print(f"行 {i+1}:信頼度スコア:{line.confidence}:テキスト: {line.text} :")

### 行の境界ポリゴンの座標の表示
(この例では1ページ目（インデックスが"0"）のすべての行のテキストと境界ポリゴンの座標を表示)

In [ ]:
for i, line in enumerate(ocr_result.pages[0].lines):
    print(f"\n行 {i+1}: {line.text}")
    print(f"座標: {line.bounding_polygon.normalized_vertices}")

### 単語数の表示
（この例では1ページ目（インデックスが"0"）に含まれる単語数のみを表示）

In [ ]:
print(len(ocr_result.pages[0].words))

### 抽出した単語のテキストの表示
(この例では1ページ目（インデックスが"0"）のすべての単語の信頼度スコアとテキストを表示)

In [ ]:
for i, word in enumerate(ocr_result.pages[0].words):
    print(f"行 {i+1}:信頼度スコア:{word.confidence}:テキスト: {word.text} :")

### 抽出した単語の境界ポリゴンの座標の表示
(この例では1ページ目（インデックスが"0"）のすべての単語のテキストと境界ポリゴンの座標を表示)

In [ ]:
for i, word in enumerate(ocr_result.pages[0].words):
    print(f"\n単語 {i+1}: {word.text}")
    print(f"座標: {word.bounding_polygon.normalized_vertices}")